# Visualization

## Our First Streamlit App

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
 
# Make and Switch to Apps Directory
%mkdir '/content/drive/My Drive/Applied-NLP-in-the-Enterprise/apps'
%cd '/content/drive/My Drive/Applied-NLP-in-the-Enterprise/apps'

### Build Streamlit App

In [ ]:
%%writefile ner_app_agnews.py

In [ ]:
# Load libraries
import spacy
import streamlit as st
import spacy_streamlit
import numpy as np
import pandas as pd
import random
 
# Set title
st.title(':star: AG News Dataset')
 
# Define function to read data
@st.cache
def read_data(file):
    read_path = '/content/drive/My Drive/Applied-NLP-in-the-Enterprise'
    data = pd.read_csv(read_path+file)
    return data
 
# Read data
data = read_data('/data/ag_dataset/prepared/train_prepared.csv')

In [ ]:
# Define function to select category
@st.cache
def select_category(data, category_option):
    return data.loc[data.class_name==category_option]
 
# Set up sidebar widgets
st.sidebar.header("Parameters")
display_selections = st.sidebar.multiselect(
 "Which data would you like to display?",
 ["Full Data","Single Article"], None)
 
category_option = st.sidebar.radio(
 'Which category would you like to explore?',
 data.class_name.unique())
 
article_option = st.sidebar.number_input(
 'Which article would you like to explore?',
 data.loc[data.class_name==category_option].index.min(),
 data.loc[data.class_name==category_option].index.max(),
                            data.loc[data.class_name
                                     ==category_option].index.min())

In [ ]:
# Set display behavior for data
if "Full Data" in display_selections:
    st.header("Full Data")
    st.write(select_category(data,category_option
                             .loc[:,["title","description"]],
                             width=1980, height=200)

In [ ]:
if "Single Article" in display_selections:
    st.header("Single Article")
    st.subheader("Title")
    st.write(data.loc[article_option,"title"])
    st.subheader("Description")
    st.write(data.loc[article_option,"description"])

In [ ]:
# Set text
default_text = data.loc[article_option,"description"]
 
# Show NER Results
st.header("NER Results")
base, custom = st.beta_columns(2)
 
# Base SpaCy Model
with base:
    base_model = spacy_streamlit.load_model("en_core_web_lg")
    doc_base = base_model(default_text)
    ner_labels = ["ORG","PERSON","GPE"]
    show_table = True
    title = "Base SpaCy Model"
    sidebar_title = "Base SpaCy Model"
    spacy_streamlit.visualize_ner(doc_base,
                                  labels=ner_labels,
                                  show_table=show_table,
                                  title=title,
                                  sidebar_title=sidebar_title)

In [ ]:
# Custom SpaCy Model
with custom:
    custom_ner_model = spacy_streamlit.load_model(
    '/content/drive/My Drive/Applied-NLP-in-the-Enterprise/
    models/ag_dataset/ner-base-V3/model-best')
    doc_custom = custom_ner_model(default_text)
    ner_labels = ["ORG","PERSON","GPE","TICKER"]
    show_table = True
    title = "Custom SpaCy Model"
    sidebar_title = "Custom SpaCy Model"
    spacy_streamlit.visualize_ner(doc_custom,
                                  labels=ner_labels, 
                                  show_table=show_table,
                                  title=title,
                                  sidebar_title=sidebar_title)

### Deploy Streamlit App

In [ ]:
# Import libraries
'''Main Libraries'''
import numpy as np
import pandas as pd
 
# Install spacy
!pip install -U spacy[cuda100]
!pip install -U spacy-lookups-data
!pip install cupy-cuda100==7.3.0
 
# Download pretrained language model (core model)
!python -m spacy download en_core_web_lg
 
# Install spacy-streamlit
!pip install spacy-streamlit

In [ ]:
# Install Streamlit
!pip install streamlit
 
# Install ngrok
!pip install pyngrok

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Define deploy function
def deploy(path):
  # Kill App
    try:
        ngrok.kill()
        print("All ngrok jobs aborted.")
    except:
        print("No ngrok jobs to kill.")

    # Set App Location
    app_location = path

    # Import NGROK
    from pyngrok import ngrok

    # Setup a tunnel to the streamlit port 80
    public_url = ngrok.connect(port='80')
    print(public_url)

!streamlit run $app_location --server.port 80 >/dev/null
 
# Deploy app
deploy('/content/drive/"My Drive"/Applied-NLP-in-the-Enterprise/\
 apps/ner_app_agnews.py')

### Explore Streamlit Web App

### Build and Deploy Streamlit App for Custom NER

In [ ]:
%%writefile ner_app_custom.py
# Load libraries
import spacy
import streamlit as st
import spacy_streamlit
import numpy as np
import pandas as pd
import random
 
# Set title
st.title(':star: Custom NER')
st.header("Custom Text")
default_text = st.text_area("Enter text to analyze.",
                            height=250)
 
# Show NER Results
st.header("NER Results")
base, custom = st.beta_columns(2)
 
# Base SpaCy Model
with base:
    base_model = spacy_streamlit.load_model("en_core_web_lg")
    doc_base = base_model(default_text)
    ner_labels = ["ORG","PERSON","GPE"]
    show_table = True
    title = "Base SpaCy Model"
    sidebar_title = "Base SpaCy Model"
    spacy_streamlit.visualize_ner(doc_base,
                                  labels=ner_labels,
                                  show_table=show_table,
                                  title=title,
                                  sidebar_title=sidebar_title)

In [ ]:
# Custom SpaCy Model
with custom:
    custom_ner_model = spacy_streamlit.load_model(
    '/content/drive/My Drive/Applied-NLP-in-the-Enterprise/
    models/ag_dataset/ner-base-V3/model-best')
    doc_custom = custom_ner_model(default_text)
    ner_labels = ["ORG","PERSON","GPE","TICKER"]
    show_table = True
    title = "Custom SpaCy Model"
    sidebar_title = "Custom SpaCy Model"
    spacy_streamlit.visualize_ner(doc_custom,
                                  labels=ner_labels,
                                  show_table=show_table,
                                  title=title,
                                  sidebar_title=sidebar_title)

In [ ]:
# Deploy NER App for Custom Text
deploy('/content/drive/"My Drive"/Applied-NLP-in-the-Enterprise/\
 apps/ner_app_agnews.py')

### Build and Deploy Streamlit App for Text Classification on AGNews Dataset

In [ ]:
%%writefile textcat_app_agnews.py
# Load libraries
import spacy
import streamlit as st
import spacy_streamlit
import numpy as np
import pandas as pd
import random
 
# Set title
st.title(':star: AG News Dataset')
 
# Define function to read data
@st.cache
def read_data(file):
    read_path = '/content/drive/My Drive/Applied-NLP-in-the-Enterprise'
    data = pd.read_csv(read_path+file)
    return data
 
# Read data
data = read_data('/data/ag_dataset/prepared/train_prepared.csv')
 
# Define function to select category
@st.cache
def select_category(data, category_option):
    return data.loc[data.class_name==category_option]
 
# Set up sidebar widgets
st.sidebar.header("Parameters")
display_selections = st.sidebar.multiselect(
 "Which data would you like to display?",
 ["Full Data","Single Article"], None)
 
category_option = st.sidebar.radio(
 'Which category would you like to explore?',
 data.class_name.unique())
 
article_option = st.sidebar.number_input(
 'Which article would you like to explore?',
 data.loc[data.class_name==category_option].index.min(),
 data.loc[data.class_name==category_option].index.max(),
 data.loc[data.class_name==category_option].index.min())
 
# Set display behavior for data
if "Full Data" in display_selections:
    st.header("Full Data")
    st.write(select_category(data,category_option)
             .loc[:,["title","description"]],
             width=1980, height=200)
 

In [ ]:
if "Single Article" in display_selections:
    st.header("Single Article")
    st.subheader("Title")
    st.write(data.loc[article_option,"title"])
    st.subheader("Description")
    st.write(data.loc[article_option,"description"])

# Set text
default_text = data.loc[article_option,"description"]
 
# Custom SpaCy Model
custom_model = spacy_streamlit.load_model(\
'/content/drive/My Drive/Applied-NLP-in-the-Enterprise/\
models/ag_dataset/textcat-prodigy-V3-base-full/model-best')
doc= custom_model(default_text)
title = "Text Classification"
spacy_streamlit.visualize_textcat(doc, title=title)
prediction = max(doc.cats, key=lambda key: doc.cats[key])
confidence = str(np.round(doc.cats[prediction],2))
st.header("Prediction: " + prediction)
st.subheader("Confidence: " + confidence)

In [ ]:
# Deploy Text Classification App for AGNews
deploy('/content/drive/"My Drive"/Applied-NLP-in-the-Enterprise/\
 apps/textcat_app_agnews.py')

### Build and Deploy Streamlit App for Text Classification on Custom Text

In [ ]:
%%writefile textcat_app_custom.py
# Load libraries
import spacy
import streamlit as st
import spacy_streamlit
import numpy as np
import pandas as pd
import random
 
# Set title
st.header(":star: Text Classification")
default_text = st.text_area("Enter text to analyze.")
 
# Custom SpaCy Model
custom_model = spacy_streamlit.load_model(\
'/content/drive/My Drive/Applied-NLP-in-the-Enterprise/\
models/ag_dataset/textcat-prodigy-V3-base-full/model-best')
doc= custom_model(default_text)
title = "Text"
spacy_streamlit.visualize_textcat(doc, title=title)
prediction = max(doc.cats, key=lambda key: doc.cats[key])
confidence = str(np.round(doc.cats[prediction],2))
st.header("Prediction: " + prediction)
st.subheader("Confidence: " + confidence)

## Conclusion